In [ ]:
!pip install --quiet pytorch-lightning==1.2.5 tqdm==4.59.0

In [ ]:
from collections import defaultdict

import random
import numpy as np
import pandas as pd
import pytorch_lightning as pl

import torch
from torch.utils.data import DataLoader, Dataset

import seaborn as sns
from matplotlib import rc
from pylab import rcParams
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

In [ ]:
# Set some plotting parameters
%matplotlib inline
sns.set_style(style='whitegrid')
rcParams['figure.figsize'] = 12, 8
tqdm.pandas()

# Generate a time series dataset
We want to generate a time series based on a N-ordered Markov process, or more precisely, on a differential equation encoding a big lag. This is the equivalent of a system with some short-term and some long-term dynamics, or a fast and slow poles.

In [ ]:
FEATURES = [f'x_{10-k}' for k in range(10)]
u = 0.1 * np.random.randn(10_000).astype(np.float32)

def generate_time_series(u) -> pd.DataFrame:
    # Parameters here are hard-coded. For the sake of the experiment, this works fine for now.
    X = np.zeros(10_000, dtype=np.float32)
    
    for k in range(10, len(X)):
        X[k] = 0.9 * X[k-1] - 0.4 * X[k-2] + 0.4 * X[k-10] + u[k]

    return X

In [ ]:
def rescale(X: np.ndarray, lower=-1.0, upper=1.0) -> np.ndarray:
    # Warning: Technically, this is a tiny bit dirty here, as we normally should only rescale for
    # values gleaned from the test-set only. However, in our little toy example, this helps have
    # the code a bit cleaner and more readable than having to do a train-test-split on X and then
    # rescale both separately and then create sequences from that.
    X_std = (X - X.min()) / (X.max() - X.min())
    return X_std * (upper - lower) + lower

In [ ]:
def generate_measurements_from_timeline(X, sequence_length=10):
    # Creates a DataFrame out of the hidden states and the measurements:
    return pd.DataFrame(
        [X[k:k+sequence_length+1] for k in range(len(X) - sequence_length)],
        columns=FEATURES + ['y']
    )

In [ ]:
# Generate measurements and internal states and randomly shuffle them:
d = generate_measurements_from_timeline(rescale(generate_time_series(u)))

Note that d now contains row-wise the sequences we want to input to the LSTM.
For our little toy example, the LSTM will be effectively working in a univariate version. For a multivariate version, we would have to adjust the code such that the we would generate a matrix with M variables and N lines of measurements of those variables.

In [ ]:
# Create sequences from those measurements in a way that Pytorch likes it ... :
def create_sequences(d: pd.DataFrame, measurement_col: str, target_col: str, sequence_length: int) -> list:
    sequences = []

    for i in tqdm(range(len(d) - sequence_length)):
        sequence = d.loc[i:i+sequence_length, measurement_col]
        label = d.loc[i+sequence_length, target_col]
        sequences.append((sequence, label))

    return sequences

In [ ]:
sequences = create_sequences(d, 'x_1', 'y', 10)

In [ ]:
# NOW randomly shuffle them!
random.shuffle(sequences)

In [ ]:
# Assign sequences to train and test sets:
N_test = int(0.9 * len(sequences))
train_sequences, test_sequences = sequences[:N_test], sequences[N_test:]

# Define Pytorch Dataset and Dataloader classes

In [ ]:
class TimelineDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence, target = self.sequences[idx]
        # Return a dictionary with sequence and target as keys.
        # As we are handling a single-input case here, we have to unsqueeze
        # in order to add a dimension such that we have a 3D-tensor in the
        # end, which is what the more general multi-variate case expects.
        return dict(
            sequence=torch.Tensor(sequence.to_numpy(dtype=np.float32)).unsqueeze(dim=1),
            target=torch.tensor(target.astype(np.float32))
        )

In [ ]:
class TimelineDatamodule(pl.LightningDataModule):
    def __init__(self, train_sequences, test_sequences, batch_size=10):
        self.train_sequences = train_sequences
        self.test_sequences = test_sequences
        self.batch_size = batch_size
        self.prepare_data()
        self.setup()

    def setup(self):
        self.train_dataset = TimelineDataset(self.train_sequences)
        self.test_dataset = TimelineDataset(self.test_sequences)

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=2,
        )
  
    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=1,
        )
  
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=1,
        )

In [ ]:
# Instantiate the data module:
BATCH_SIZE = 32

data_module = TimelineDatamodule(train_sequences, test_sequences, BATCH_SIZE)

# Define Pytorch model

In [ ]:
class TimelineLSTMModule(torch.nn.Module):

    def __init__(self, input_size: int, hidden_size: int, num_layers: int, dropout: float=0.2):
        super().__init__()

        self.hidden_size = hidden_size

        self.lstm = torch.nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            batch_first=True,
            num_layers=num_layers,
            dropout=dropout,
        )

        self.regressor = torch.nn.Linear(in_features=hidden_size, out_features=1)

    def forward(self, x):
        # Flatten parameters for better GPU-memory-usage in distributed training
        # (good practice, though not really needed here):
        self.lstm.flatten_parameters()

        _, (lstm_hidden, _) = self.lstm(x)

        return self.regressor(lstm_hidden[-1])

In [ ]:
class TimelineLSTMModel(pl.LightningModule):

    def __init__(self, input_size: int, hidden_size: int, num_layers: int, dropout: float=0.2):
        super().__init__()

        self.model = TimelineLSTMModule(
            input_size=input_size, hidden_size=hidden_size, 
            num_layers=num_layers, dropout=dropout
        )
        self.criterion = torch.nn.MSELoss()

    def forward(self, x, target=None):
        output = self.model(x)
        loss = 0
        if target is not None:
            loss = self.criterion(output, target.unsqueeze(dim=1))
        return loss, output

    def training_step(self, batch, batch_idx):
        loss, output = self(batch['sequence'], batch['target'])
        self.log('train_loss', loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, output = self(batch['sequence'], batch['target'])
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        loss, output = self(batch['sequence'], batch['target'])
        self.log('test_loss', loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=1e-4)

In [ ]:
model = TimelineLSTMModel(input_size=1, hidden_size=64, num_layers=2, dropout=0.2)

# Training and logging

In [ ]:
# Set up tensorboard to monitor the trainings:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

In [ ]:
# Define the callbacks, logger and trainer:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath='checkpoints',
    verbose=True,
    save_top_k=1,
    filename='best_model',
    monitor='val_loss',
    mode='min',
)

early_stopping_callback = pl.callbacks.EarlyStopping('val_loss', patience=5, verbose=True, mode='min')

logger = pl.loggers.TensorBoardLogger('lightning_logs', name='timeline_prediction')

trainer = pl.Trainer(
    checkpoint_callback=checkpoint_callback,
    callbacks=[early_stopping_callback],
    logger=logger,
    max_epochs=250,
    gpus=1,
    progress_bar_refresh_rate=30,
)

In [ ]:
trainer.fit(model, datamodule=data_module)

# Analyse model results

In [ ]:
trained_model = TimelineLSTMModel.load_from_checkpoint('./checkpoints/best_model.ckpt', input_size=1, hidden_size=64, num_layers=2)
trained_model.freeze()

In [ ]:
test_dataset = TimelineDataset(test_sequences)

In [ ]:
predictions = []
targets = []

for t in tqdm(test_dataset):
    _, output = trained_model(t['sequence'].unsqueeze(dim=0))
    predictions.append(output.item())
    targets.append(t['target'].tolist())

In [ ]:
# Generate a dataframe to compare predictions and targets:
res = pd.DataFrame(
    {'predictions': predictions, 'targets': targets}
)

# Order that dataframe such that the targets are ascending,
# to get a cleaner view:
res = res.sort_values('targets').reset_index(drop=True)

In [ ]:
res.plot(y=['targets', 'predictions'])

Given that the model does not know the random-disturbance $u$, the result is as expected: The general trend is caught. All is working fine.